In [1]:
from kanon.units import Sexagesimal
import astropy.table
from kanon.tables import HTable
import math

In [2]:
# For these calculations we will round after every operations

from kanon.units.precision import get_context, TruncatureMode, PrecisionMode
get_context().mutate(tmode=TruncatureMode.ROUND,pmode=PrecisionMode.MAX)


In [3]:
OBLIQUITY = "23;51,20"

In [4]:
# We construct a table of sine from 0 to 90

sin_table = HTable([
    list(Sexagesimal.range(91)),
    [round(Sexagesimal.from_float(math.sin(x * math.pi / 180), 3)) for x in range(91)]
], names=("Arg", "Val"), index="Arg")

sin_table

# We apply a multiplication on all the sine values to get obl_table = Sin(x) * obl

obl = sin_table.get(Sexagesimal(OBLIQUITY))

obl_table = sin_table.apply("Val", lambda x: x * obl)
obl_table


Arg,Val
object,object
00 ;,"00 ; 00,00,00"
01 ;,"00 ; 00,25,25"
02 ;,"00 ; 00,50,49"
03 ;,"00 ; 01,16,12"
04 ;,"00 ; 01,41,33"
05 ;,"00 ; 02,06,54"
06 ;,"00 ; 02,32,11"
07 ;,"00 ; 02,57,26"
08 ;,"00 ; 03,22,37"


In [5]:
# Arcsin is sin_table taken from Val to Arg instead of Arg to Val

arcsin_table = sin_table.copy(set_index="Val")

# For example, if we want arcsin(0.3) :

arcsin_table.get(0.3)


17 ; 27,29,49,59

In [6]:
# Finally, we apply arcsin_table.get on every obl_table values to find decl_table = arcsin(obltable)

decl_table = obl_table.apply("Val", arcsin_table.get)
decl_table

Arg,Val
object,object
00 ;,00 ;
01 ;,"00 ; 24,16,14"
02 ;,"00 ; 48,31,31"
03 ;,"01 ; 12,46,15"
04 ;,"01 ; 36,59,26"
05 ;,"02 ; 01,12,38"
06 ;,"02 ; 25,22,23"
07 ;,"02 ; 49,30,13"
08 ;,"03 ; 13,35,19"


In [7]:
# Let's make the same calculation with an interpolated sine table

# First, extract every multiple of 6 from the sine table

sin_table_grid_6 = sin_table[sin_table["Arg"] % 6 == 0]

sin_table_grid_6

Arg,Val
object,object
00 ;,"00 ; 00,00,00"
06 ;,"00 ; 06,16,18"
12 ;,"00 ; 12,28,29"
18 ;,"00 ; 18,32,28"
24 ;,"00 ; 24,24,15"
30 ;,"00 ; 30,00,00"
36 ;,"00 ; 35,16,02"
42 ;,"00 ; 40,08,52"
48 ;,"00 ; 44,35,19"


In [8]:
# We can populate it with its interpolation method

sin_table_pop_6 = sin_table_grid_6.populate(list(Sexagesimal.range(91)), method="interpolate")

sin_table_pop_6

Arg,Val
object,object
00 ;,"00 ; 00,00,00"
01 ;,"00 ; 01,02,43"
02 ;,"00 ; 02,05,26"
03 ;,"00 ; 03,08,09"
04 ;,"00 ; 04,10,52"
05 ;,"00 ; 05,13,35"
06 ;,"00 ; 06,16,18"
07 ;,"00 ; 07,18,20"
08 ;,"00 ; 08,20,22"


In [9]:
# Comparing the 2nd difference between the true sine table and the interpolated one

true_2diff = sin_table.diff(n=2, new_name="True Sine 2-diff")
interpolated_2diff = sin_table_pop_6.diff(n=2, new_name="Interpolated Sine 2-diff")


astropy.table.join(true_2diff, interpolated_2diff)

Arg,True Sine 2-diff,Interpolated Sine 2-diff
object,object,object
00 ;,"00 ; 00,00,00","00 ; 00,00,00"
01 ;,"00 ; 01,02,50","00 ; 01,02,43"
02 ;,"-00 ; 00,00,02","00 ; 00,00,00"
03 ;,"-00 ; 00,00,01","00 ; 00,00,00"
04 ;,"-00 ; 00,00,05","00 ; 00,00,00"
05 ;,"-00 ; 00,00,03","00 ; 00,00,00"
06 ;,"-00 ; 00,00,07","00 ; 00,00,00"
07 ;,"-00 ; 00,00,06","-00 ; 00,00,41"
08 ;,"-00 ; 00,00,09","00 ; 00,00,00"


In [ ]:
# With this newly built table we are able to construct a new declination table

obl_interp = sin_table_pop_6.get(Sexagesimal(OBLIQUITY))

obl_table_interp = sin_table_pop_6.apply("Val", lambda x: x * obl)
arcsin_table_interp = sin_table_pop_6.copy(set_index="Val")

obl_table_interp.apply("Val", arcsin_table_interp.get)

